##### To get the word count, we ran this in bash:

$SPARK_HOME/bin/spark-submit ./word_count.py \
	--num_output_partitions 1 --log_level WARN \
	./input/test_wet.txt wordcounts

##### To get the server names, we ran this in bash:

$SPARK_HOME/bin/spark-submit ./server_count.py \
	--num_output_partitions 1 --log_level WARN \
	./input/test_warc.txt servernames
    
##### To get links, we ran this in bash:

$SPARK_HOME/bin/spark-submit ./wat_extract_links.py \
	--num_output_partitions 1 --log_level WARN \
	./input/test_wat.txt links

In [19]:
import pyspark as ps
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import VectorAssembler, MinMaxScaler, RegexTokenizer, HashingTF, PCA
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder

from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import ujson as json

In [20]:
spark = ps.sql.SparkSession.builder.master("local")\
                                    .appName("web_crawl")\
                                    .getOrCreate()

In [21]:
servercounts = spark.read.parquet('servernames')
servernames.printSchema()

wordcounts = spark.read.parquet('wordcounts')
wordcounts.printSchema()

linkextract = spark.read.parquet('links')
linkextract.printSchema()

root
 |-- key: string (nullable = true)
 |-- val: long (nullable = true)



root
 |-- key: string (nullable = true)
 |-- val: struct (nullable = true)
 |    |-- tf: long (nullable = true)
 |    |-- df: long (nullable = true)



In [25]:
wordcounts.show()

+--------------------+----------+
|                 key|       val|
+--------------------+----------+
|         47207221вск|    [1, 1]|
|         деструкторы|    [1, 1]|
|               سادات|    [8, 8]|
|              werner|[190, 119]|
|              τραύμα|    [2, 2]|
|             acortar|    [4, 4]|
|         unbelievers|    [9, 7]|
|       transcription|[279, 106]|
|          spacewalks|    [1, 1]|
|             อสำรองท|    [1, 1]|
|            طحانشاطر|    [1, 1]|
|             三天两头出疑问|    [1, 1]|
|              hauser|  [56, 26]|
|          empfundene|    [1, 1]|
|        実際の大きさは１mm程度|    [1, 1]|
|先生ほどの人でも少しはおこまりでしょうな|    [1, 1]|
|            trotters|  [15, 13]|
|             morsels|  [17, 13]|
|               izumi|  [46, 33]|
|            andrew61|    [2, 1]|
+--------------------+----------+
only showing top 20 rows

